In [1]:
#Importación de librerías

# Usada para la importación de los datos
import pandas as pd
# Usada para los cáculos
import numpy as np

# Para ver la similitud entre dos registros
from sklearn.metrics.pairwise import cosine_similarity
# Para crear una matriz dispersa
from scipy.sparse import csr_matrix
# Para los cálculos desde la matriz dispersa
from scipy.sparse.linalg import svds

# Calculo de KNN y accuracy
from surprise import KNNWithMeans, Dataset, accuracy, Reader
# Para separar los datos del entrenamiento
from surprise.model_selection import train_test_split

# Importación de los datos

In [2]:
#Función para importar los datos
def data():
    df = pd.read_csv('data.csv')
    df.shape
    print("Dataset \n",df.head())
    return df

In [3]:
df=data()

Dataset 
    book_id  author_id  book_genre  reader_id  num_pages  book_rating  \
0      655         52           4      11482        300            4   
1     2713         90           3       6479        469            1   
2      409         17           2      25472        435            1   
3     1150        234          10      23950        529            2   
4     2424        390           5      13046        395            2   

   publisher_id  publish_year  book_price  text_lang  
0             8          2012          94          7  
1             8          2012          33          5  
2            12          2001         196          4  
3            23          2019          79          2  
4            20          2010         200          4  


# Método 1

En este primer método se hace las recomendaciones a un usuario dependiendo las calificiones de otros usuarios.

In [4]:
# generar una tabla dinámica con lectores en el índice y libros en la columna y los valores son las calificaciones
def dinamitable(df):
    pt_df = df.pivot_table(
        columns = 'book_id',
        index = 'reader_id',
        values = 'book_rating'
    ).fillna(0)

    # convertir a una matriz csr
    mat = pt_df.values
    mat = csr_matrix(mat)
    return pt_df, mat

In [5]:
# Función para normalizar la entrada de pred_ratings
def normalize(pred_ratings):
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

In [6]:
# Función que calcula la descomposición del valor de la entrada de la matriz, también
# genera y normaliza las predicciones de la calificación de los usuarios
def generate_prediction_df(mat, pt_df, n_factors):
    
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # Factorización de matriz
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # Caclula la predicción de los ratings
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    
    # Convierte a df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df, pred_ratings

In [7]:
# Función que dado el id del usuario y pred_df hará recomendaciones al usuario
# Los parámetros son: pred_df que se generó en generate_prediction_df, el id del usuario, y el número de recomendaciones
def recommend_items(pred_df, usr_id, n_recs):    
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    print("\n Recomendaciones para el usuario con id:",usr_id,"\n", rec_df)

In [8]:
# Ejecución del código método 1
df=data()
pt_df, mat=dinamitable(df)
pred_df, pred_ratings = generate_prediction_df(mat, pt_df, 10)
normalize(pred_ratings)
rec_df=recommend_items(pred_df, 5, 5)

Dataset 
    book_id  author_id  book_genre  reader_id  num_pages  book_rating  \
0      655         52           4      11482        300            4   
1     2713         90           3       6479        469            1   
2      409         17           2      25472        435            1   
3     1150        234          10      23950        529            2   
4     2424        390           5      13046        395            2   

   publisher_id  publish_year  book_price  text_lang  
0             8          2012          94          7  
1             8          2012          33          5  
2            12          2001         196          4  
3            23          2019          79          2  
4            20          2010         200          4  

 Recomendaciones para el usuario con id: 5 
    book_id       sim
0     1108  0.143244
1     1839  0.140945
2      725  0.140516
3     1897  0.140497
4     1341  0.140086


# Método 2

En este segundo método se hace las recomendaciones de otros lectores similares a un lector.

In [9]:
# Crea una estructura.
def estruc(df):
    pt_df = df.pivot_table(
        columns = 'book_id', 
        index = 'author_id', 
        values = 'book_rating'
    ).fillna(0)
    return pt_df

In [10]:
# Función para encontrar lectores similares al lector pasado
def find_similar_readers(pt_df, reader_id, n_recs):
    
    # separación del lector de los demás lectores
    reader = pt_df[pt_df.index == reader_id]
    other_readers = pt_df[pt_df.index != reader_id]

    # obteniendo lectores similares a otros lectores
    sim = cosine_similarity(reader, other_readers)[0].tolist()
    idx = other_readers.index.tolist()

    # crea un diccionario similar al de otros lectores
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)

    similar_readers = idx_sim[:n_recs]
    readers = [rdr[0] for rdr in similar_readers]

    print("\n Lectores similares, al lector con id:",reader_id," \n", readers)

In [11]:
# Ejecución del código método 2
df=data()
pt_df=estruc(df)
similar_readers=find_similar_readers(pt_df = pt_df, reader_id = 226, n_recs = 5)


Dataset 
    book_id  author_id  book_genre  reader_id  num_pages  book_rating  \
0      655         52           4      11482        300            4   
1     2713         90           3       6479        469            1   
2      409         17           2      25472        435            1   
3     1150        234          10      23950        529            2   
4     2424        390           5      13046        395            2   

   publisher_id  publish_year  book_price  text_lang  
0             8          2012          94          7  
1             8          2012          33          5  
2            12          2001         196          4  
3            23          2019          79          2  
4            20          2010         200          4  

 Lectores similares, al lector con id: 226  
 [319, 191, 145, 162, 212]


# Método 3

Se hace la recomendación basado por valores boleanos

In [12]:
#Selección de los campos
def dataset(df):
    rdf = df[['reader_id', 'book_id', 'book_rating']]
    # Lectura del dataset
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(rdf, reader)
    return rdf, reader, data

In [13]:
#Separación de los datos de entrenamiento
def splitdata(data):
    trainset, testset = train_test_split(data, test_size=0.3,random_state=10)
    return trainset, testset

In [14]:
# Función basado en el usuario, cambia entre filtrado colaborativo o basado en elementos mediante V o F
def mode(trainset):
    algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo.fit(trainset)
    return algo

In [15]:
# Función para ejecutar el modelo
def modtest(algo):
    test_pred = algo.test(testset)
    return test_pred

In [16]:
# Función para las métricas
def metrics(test_pred):
    # get RMSE
    print(accuracy.mse(test_pred, verbose=True))
    print(accuracy.rmse(test_pred, verbose=True))
    print(accuracy.mae(test_pred, verbose=True))

In [17]:
def predi(id_usuario, iid):
    print(" \n La predicción para el usuario con id:",id_usuario," tiene: \n",algo.predict(uid = id_usuario, iid = iid))

In [18]:
# Ejecución del código método 3
df=data()
rdf, reader, data=dataset(df)
trainset, testset=splitdata(data)
algo=mode(trainset)
test_pred=modtest(algo)
metrics(test_pred)
predi(id_usuario=10, iid=43)

Dataset 
    book_id  author_id  book_genre  reader_id  num_pages  book_rating  \
0      655         52           4      11482        300            4   
1     2713         90           3       6479        469            1   
2      409         17           2      25472        435            1   
3     1150        234          10      23950        529            2   
4     2424        390           5      13046        395            2   

   publisher_id  publish_year  book_price  text_lang  
0             8          2012          94          7  
1             8          2012          33          5  
2            12          2001         196          4  
3            23          2019          79          2  
4            20          2010         200          4  
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 8.5885
8.588525215348367
RMSE: 2.9306
2.9306185721359865
MAE:  2.5090
2.509045670996971
 
 La predicción 